# Dielectric Properties

In this notebook, we will investigate the varying backscatter values associated with different land surfaces like water bodies, forests, grasslands and urban areas. We will use backscatter data from the Sentinel-1 satellite and we will utilize the CORINE Land Cover dataset to classify and extrapolate these surfaces, enabling us to analyze how different land cover types influence backscatter responses.

In [ ]:
import json

import holoviews as hv
import intake
import numpy as np
import rioxarray  # noqa: F401
import xarray as xr
from matplotlib.colors import BoundaryNorm, ListedColormap

from mrs.catalog import CorineColorCollection, get_intake_url
from mrs.plot import plot_corine_data, plot_variability_over_time

hv.extension("bokeh")

## Load Sentinel-1 Data

For our analysis we are using sigma naught backscatering data from Sentinel-1. The images we are analyzing cover the region south of Vienna and west of Lake Neusiedl. We load the data and and apply again a preprocessing function. Here we extract the scaling factor and the date the image was taken from the metadata. We will focus our attention to a smaller area containing a part of the Lake Neusiedl Lake and its surrounding land. The obtained`xarray` dataset and is then converted to an array, because we only have one variable, the VV backscatter values.


In [ ]:
url = get_intake_url()
cat = intake.open_catalog(url)
sig0_da = cat.neusiedler.read().sig0.compute()

Let's have a look at the data by plotting the first timeslice.


In [ ]:
sig0_da.isel(time=0).plot(robust=True, cmap="Greys_r").axes.set_aspect("equal")

## Load CORINE Landcover Data

We will load the CORINE Land Cover, which is a pan-European land cover and land use inventory with 44 thematic classes. The resolution of this classification is 100 by 100m and the file was created in 2018
([CORINE Land Cover](https://land.copernicus.eu/en/products/corine-land-cover)).


In [ ]:
cor_da = cat.corine.read().land_cover.compute()

### Colormapping and Encoding

For the different land cover types we use the official color encoding which can be found in [CORINE Land Cover](https://collections.sentinel-hub.com/corine-land-cover/readme.html).


In [ ]:
# Load encoding and validate with Pydantic
with cat.corine_cmap.read()[0] as f:
    corine_colors = CorineColorCollection.model_validate(json.load(f))

# Create cmap and norm for plotting
colors = [info["color"].as_hex() for info in corine_colors.items]
categories = [info["value"] for info in corine_colors.items]
cmap = ListedColormap(colors)
norm = BoundaryNorm([*categories, max(categories) + 1], len(categories))

Now we can plot the CORINE Land Cover dataset.


In [ ]:
present_landcover_codes = np.unique(cor_da.values[~np.isnan(cor_da.values)].astype(int))

# Create the plot
plot_corine_data(cor_da, cmap, norm, corine_colors, present_landcover_codes)

Now we are ready to merge the backscatter data (`sig0_da`) with the land cover dataset (`cor_da`) to have one dataset combining all data.


In [ ]:
var_ds = xr.merge([sig0_da, cor_da]).drop_vars("band")
var_ds

## Backscatter Variability

With this combined dataset we can study backscatter variability in relation to natural media. For example we can look at the backscatter variability for water by clipping the dataset to only contain the land cover class water, like so:


In [ ]:
# 41 = encoded value for water bodies
encoded_value_for_waterbodies = 41
waterbodies_mask = var_ds.land_cover == encoded_value_for_waterbodies
waterbodies_mask.plot().axes.set_aspect("equal")

This gives use backscatter values over water only.


In [ ]:
waterbodies_sig0 = var_ds.sig0.isel(time=0).where(waterbodies_mask)
waterbodies_sig0.plot(robust=True, cmap="Greys_r").axes.set_aspect("equal")

To get an idea of the variability we can create a histogram. Radar backscatter from water bodies fluctuates with surface roughness, which changes with wind conditions, creating spatial and temporal variations in signal intensity.


In [ ]:
waterbodies_sig0.plot.hist(bins=50, edgecolor="black")

## Variability over Time

Next we will look at the changes in variability in backscatter values over time for each of the CORINE Land Cover types. We do this by creating the following interactive plot. We can spot that backscatter in agricultural fields varies due to seasonal cycles like planting, growing, and harvesting, each of which changes vegetation structure. Changes in backscatter are strongly related to soil moisture content from irrigation or rainfall. Ultimately, phenological stages of crops and canopy moisture dynamics can affect the backscatter signal.


In [ ]:
plot_variability_over_time(corine_colors.to_dict(), var_ds, present_landcover_codes)